In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install autocorrect

In [1]:
import re, os, sys, nltk
import xml.etree.cElementTree as et
import pickle, base64, time
from heapq import *
import math, operator, json, copy
from autocorrect import Speller
spell = Speller()
# basePath = '/content/drive/MyDrive/Wiki-Search/'

In [2]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1
#print(stop_words)

In [4]:
fields = {}
fields["t"] = open("title_output.txt","r")
fields["c"] = open("category_output.txt","r")
fields["i"] = open("infobox_output.txt","r")
fields["b"] = open("body_text_output.txt","r")

In [5]:
fields['c']

<_io.TextIOWrapper name='category_output.txt' mode='r' encoding='UTF-8'>

In [6]:
words_position = {}
with open("word_position.json",'r') as f:
    words_position = json.load(f)

In [7]:
title_position = pickle.load(open("title_position.pickle", "rb"))

In [8]:
titles = open("titles.txt",'r',encoding="utf8")
results = 10

In [ ]:
while True:
    query_result = []
    query = input("> ")
    if (query == "quit") :
        break
    start = time.time()
    result_doc = {}
    if ":" in query:
        query_fields = query.split(" ")
        for queries in query_fields:
            field, query = queries.split(":")
            query = spell(query)
            print("Field Query after auto correction: ", query)
            if field == "ref" or field == "ext" or field == "body":
                field = "b"
            elif field == "title":
                field = "t"
            elif field == "category":
                field = "c"
            elif field == "infobox":
                field = "i"
            words = query.split(" ")
            for word in words:
                word = stemmer.stem(word)
                if word and word not in stop_words:
                    if word in words_position and field in words_position[word]:
                        docs = []
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[:-1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            print(doc)
                            doc_no, score = doc.split(":")
                            if doc_no not in result_doc:
                                result_doc[doc_no] = 0
                            result_doc[doc_no] += float(score)
    else:
        query = spell(query)
        print("Normal Query after auto-correction: ", query)
        query_words = query.split(" ")
        for word in query_words:
            word = word.strip()
            word = stemmer.stem(word)
            if word not in stop_words:
                if word in words_position:
                    docs = []
                    for field in words_position[word].keys():
                        pointer = words_position[word][field]
                        print(field)
                        fields[field].seek(pointer)
                        posting_list = fields[field].readline()[: -1]
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            doc_no, score = doc.split(":")
                            if doc_no not in result_doc:
                                result_doc[doc_no] = 0
                            result_doc[doc_no] += float(score)
        
    count = 1
    result_doc = sorted(result_doc.items(), key = operator.itemgetter(1), reverse = True)
    for document in result_doc :

        position = title_position[int(document[0]) - 1]
        titles.seek(position)
        query_result.append(titles.readline()[: -1])
        if count >= results :
            break
        count += 1
        
    end = time.time()
    print("Time Taken :- ",(end-start))
    if len(query_result) == 0:
        print("No result found")
    else:
        print("Number of Results :- ",len(query_result))
        for result in query_result:
            print(result)

> Gandhi
Normal Query after auto-correction:  Gandhi
t
c
i
b
Time Taken :-  0.012713909149169922
Number of Results :-  10
Gandhi's autobiography
Autobiography Gandhi
Gandhi An Autobiography
Wikipedia:Votes for deletion/Better Than Gandhi
Gandhi (American band)
Wikipedia:Wikipedia Signpost/2005-05-16/Features and admins
Simi Garewal
Sonipat district
Pakistani Americans
One man, one vote
> Hello
Normal Query after auto-correction:  Hello
t
c
i
b
Time Taken :-  0.053797006607055664
Number of Results :-  10
Hello Kitty World
Sega Dreamcast Hello Kitty
Dreamcast Hello Kitty
Hello Deli
Hello Joe
Hello Darwin
Template:Hello
Apollo 11
ABBA
Amplitude modulation
> title:Gandhi
Field Query after auto correction:  Gandhi
1020301:-1.87
1042854:-1.87
105556:-1.87
113753:-1.87
122258:-1.87
136327:-1.87
1106487:-1.87
1126110:-1.87
1126432:-1.87
1128214:-1.87
1131116:-1.87
1131119:-1.87
1146939:-1.87
1185630:-1.87
1223790:-1.87
1237750:-1.87
1243294:-1.87
1245082:-1.87
1253343:-1.87
1277882:-1.87
12778